# Numerical Analysis (Math 445 and 545)

## Lecture 7 - Errors and Condition Numbers

**Author: Daniel Poll**
<br>
**Date: February 3, 2020**

## Motivation

In the previous lecture, we saw that solutions to the system of linear equations $Ax = b$ can be found rather quickly through a Gaussian elimination-type algorithm called LU decomposition. However, we made many assumptions in that lecture, such as $A$ being a square matrix and $A$ being nonsingular. On top of that, we saw that numerical implementation can lead to round-off error. It makes sense that we have a way to check if this may be an issue. 


Further, if we break the assumption that $A$ is $n \times n$, then we are not guaranteed a solution. In fact, we may have multiple solutions, a unique solution, or no solution at all. Most of our focus in the next few lectures will be on *overdetermined* systems. Indeed, these are systems that most students have already seen. For example, linear regression requires fitting $n$ points to an equation which only has two variables: a slope and a y-intercept. This can be phrased an overdetermined system (discussed in more detail later).

So, it's in our best interest to discuss techniques that can be used when we have such a system. 




## Matrix Errors and Norms

For an approximation $x_a$ of the solution $x$ of $A x = b$, the *residual* $r = A x_a - b$ measures error as *backward error*, often measured by a single number, the residual norm $\| A x_a - b \|$.
Any norm could be used, but the maximum norm or the 2-norm is usually preferred, for reasons that we will see soon.

The corresponding (dimensionless) measure of relative error is defined as
$$\frac{\|r\|}{\|b\|}.$$

However, these can greatly underestimate the *forward* errors in the solution: the absolute error $\|x - x_a\|$ and relative error
$$Rel(x_a) = \frac{\|x - x_a\|}{\| x \|}$$

To relate these to the residual, we need the concepts of a *matrix norm* and the *condition number* of a matrix.

These induced matrix norms have many properties in common with Euclidean length and other vector norms, but there can also be products, and then one has to be careful.

1. $\|A\| \geq 0$ (positivity)
1. $\| A \| = 0$ if and only if $A = 0$ (definiteness)
2. $\| c A \| = |c| \, \|A\|$ for any constant $c$ (absolute homogeneity)
3. $\| A + B \| \leq \| A \| + \| B \|$ (sub-additivity or the triangle inequality),
<br>
and when the product of two matrices makes sense (including matrix-vector products),
4. $\| A B \| \leq \| A \| \, \| B \|$ (sub-multiplicativity)

Note the failure to always have equality with products.
Indeed one can have $A B = 0$ with $A$ and $B$ both non-zero, such as when $A$ is a singular matrix and $B$ is a null-vector for it.

Two common choices for matrix norms:

1. Given any vector norm $\| \cdot \|$ — such as the maximum ("infinity") norm $\| \cdot \|_\infty$ or the Euclidean norm (length) $\| \cdot \|_2$ — the correponding *induced matrix norm* is
$$
\| A \| := \max_{x \neq 0} \frac{\| Ax \|}{\| x \|}, =  \max_{\|x\|=1} \| Ax \|
$$
This maximum exists for the infinity norm, and indeed there ia an explicit formula for it:
for any $m\times n$ matrix,
$$
\|A\|_\infty = \max_{i=1}^m \sum_{j=1}^n |a_{ij}|
$$
(On the other hand, it is far harder to compute the Euclidean norm of a matrix: the formula requires computing eigenvalues.)

Note that when the matrix is a vector considered as amatrix with a single column — so $n=1$ — the sum goes away, and this agrees with the infinity vector norm.
This allows us to consider vectors as being just matrices with a single column, which we will often do from now on.

2. Due to the difficulty of the induced 2-norm, a popular alternative is the *Frobenius norm*. We usually denote the Frobenius norm with a subscript $F$ and write
$$
||A||_F = \sqrt{ \sum_{i=1}^m \sum_{j=1}^n |a_{ij}|^2} = \sqrt{ \text{trace}\big(A^*A\big) }
$$
Note that this is *not* the norm that is induced by the vector 2-norm, although that may look to be the case at first glance. In fact, it can be shown that the Frobenius norm is an upper bound for the induced 2-norm. So, it is a good alternative. 

## Condition Numbers

It can be proven that, that if $A$ is square and invertible, then for any choice of norm,

$$
Rel(x_a) = \frac{\|x - x_a\|}{\| x \|} \leq \|A\|\|A^{-1}\|\frac{\|r\|}{\|b\|},
$$

where the last factor is the relative backward error. We can do this by recognizing two main points. If 

1. If $A(x-x_a) = r$, then $\|x-x_a\| \leq \|A^{-1}\|\|r\|$
2. If $Ax = b$, then $\|A\|\|x\| \geq \|b\|$. 

Combining these two inequalities will give the result.

Since we can (though often with considerable effort, due to the inverse!) compute the right-hand side when the infinity norm is used, we can compute an upper bound on the relative error.
Form this, an upper bound on the absolute error can be computed if needed.

The *growth factor* between the relative backward error measured by the residual and the relative (forward) error is called the *condition number*, $K(A)$:
$$
\kappa(A) := \|A\| \|A^{-1}\|
$$
Note that if $A$ is singular, then this undefined. So, we would generally say the condition number is infinite. On the other hand, we can quickly verify based on norm properties that
$$
\kappa(A) := \|A\|\|A^{-1}\| \geq \|A A^{-1}\| = 1
$$
So, matrices that are close to $1$ are extremely nice to have. As an example, the identity matrix has a condition number of exactly $1$, as it is its own inverse. Another example of a matrix with condition number $1$ is a permutation elementary matrix $P_{ij}$, discussed in the previous lecture. This intuitively makes sense: swapping rows has no impact on algebraic operations. 

In fact, there is one condition number for each choice of norm, so we will usually work with
$$
\kappa_\infty(A) := \|A\|_\infty \|A^{-1}\|_\infty
$$
However, there may be times we also work with the 2-norm or the Frobenius norm.

In Python, a good approximation of the condition number $\kappa_\infty(A)$ is given by `numpy.linalg.cond(A, numpy.inf)`.
<br>
As with `numpy.linalg.norm`, the simple form `numpy.linalg.cond(A)` defaults to using the 2-norm, based on the Euclidian length $\| \cdot \|_2$ for vectors.

This is not done exactly, since computing the inverse is a lot of work for large matrices and good estimates can be found far more quickly.
The basic idea is start with the formula
$$
\| A^{-1} \| = \max_{\|x\|=1} \| A ^{-1} x \|
$$
and instead compute the maximum over some finite selection of values for $x$: call them $x^{(k)}$.
Then to evaluate $y^{(k)} =  A ^{-1} x^{(k)}$, express this through the equation $A y^{(k)} = x^{(k)}$.
Once we have an LU factorization for $A$ (which one probably would have when exploring errors in a numerical solution of $Ax = b$) each of these systems can be solved relatively fast:
Then
$$
\| A^{-1} \| \approx \max_k \| y^{(k)} \|.
$$

Condition numbers, giving upper limit on the ratio of forward error to backward error,
measure the amplification of errors, and have counterparts in other contexts.
For example, with an approximation $r_a$ of a root $r$ of the equation $f(x) = 0$, the ratio of forward error to backward error is bounded by
$\displaystyle \max 1/| f'(x) | = \frac{1}{\min | f'(x) |}$, where the maximum only need be taken over an interval known to contain both the root and the approximation.
This condition number becomes "infinite" for a multiple root, $f'(r) = 0$.

For a unique solution, careful calculation of an approximate solution $x_a$ of $Ax = b$ can often get a *residual* that is at the level of machine rounding error, so that roughly the relative backward error is of size comparable to the machine unit, $u$.
The condition number then guarantees that the (forward) relative error is no greater than about $u \, \kappa(A)$.

In terms of significant bits, with $p$ bit machine arithmetic, one can hope to get $p - \log_2(\kappa(A))$ significant bits in the result, but can not rely on more, so one loses $\log_2(\kappa(A))$ significant bits.

A *well-conditioned problem* is one that is not too highly sensitive to errors in rounding or input data; for an eqution $Ax = b$, this corresponds to the condition number of $A$ not being to large; the matrix $A$ is then sometimes also called well-conditioned.
This is of course vague, but might typically mean that $p - \log_2(\kappa(A))$ is a sufficient number of significant bits for a particular purpose.

A problem that is not deemed well-conditioned is called *ill-conditioned*, so that a matrix of uncomfortably large condition number is also sometimes called ill-conditioned.
An ill-conditioned problem might still be well-posed, but just requiring careful and precise solution methods.

It's also important to note that even though a matrix is well-conditioned, it is important to consider the implementation of the algorithm for the problem at-hand. For example, returning to Gaussian elimination, consider the system of equations

$$
\begin{bmatrix}
\epsilon &1 \\
1 &1 \\
\end{bmatrix} \begin{bmatrix}
x_1 \\
x_2 \end{bmatrix} = \begin{bmatrix}
1 \\ 
0 \end{bmatrix}
$$

with $\epsilon \ll 1$. We can compute the condition number using the max norm fairly easily. We have $\kappa_\infty = 4/(1-\epsilon)$, which for $\epsilon$ small, is close to 1.

Further, we know the solution is exactly $x_1 = -x_2 = -1/(1-\epsilon) \approx -1$. However, in Gaussian elimination, we would multiply the first equation by $1/\epsilon$ and subtract, giving $(1-1/\epsilon)x_2 = -1/\epsilon$. This may be rounded to $1$ if $\epsilon$ is sufficiently small, which when plugged back in for $x_1$ would give $\epsilon x_1 = 1-1 = 0$, which is incorrect. But, this can be corrected with pivoting. 

## Overdetermined Linear System

Given a $n \times m$ matrix $A$, $n \geq m$, and an $n$-vector $b$, we seek the $m$-vector $x$ that is the best approximate solution of $Ax = b$ in the sense that the residual $Ax - b$ is as small as possible. However, we should say under what norm we wish the solution to be small: in general, this is done by minimizing the residual of the 2-norm. So, we wish to compute $\tilde{x}$ such that
$$
f(x) := \|Ax - b\|_2
$$
is a minimum. Note here that this ia the vector 2-norm. As it turns out, we can apply classic techniques from multivariable calculus to find the solution. First, find the critical points of $f$ by setting $\nabla f(x) = 0$. Then, after solving, we can show with careful manipulation that the *Hessian* matrix $\nabla^2 f$ has a strictly positive determinant, which justifies that it is a minimum. The details are left as an exercise. 

However, assuming those calculations were done, it turns out that the solution is
$$(A^* A) \tilde{x} = A^* b,$$
which is now an $m \times m$ system of equations with a unique solution — and it is the "right" one! Moreover, it can be shown that $A^* A$ is invertible as long as $A$ has independent columns.

To outline the justification:

1. Note that $A^* A$ is $m \times m$. So, we have $A^* A$ is square. 
2. $A$ and $A^* A$ share the same nullspace, as
$$
A^* Ax = 0 \iff x^* A^* A x = 0 \iff (Ax)^* Ax = 0 \iff \|Ax\|_2^2 = 0 \iff Ax = 0
$$

Since $A^*A$ is square and nonsingular, it is invertible. Thus, we define the unique solution that minimizes the least-squares residual using the *Moore-Penrose* or *pseudo-inverse* $A^+ := (A^*A)^{-1}A^*$:
$$
\tilde{x} = A^+ b 
$$

In fact, this is commonly used to compute the condition number for non-square matrices in software packages, i.e. we can extend our definition of a condition number to be the following:

$$
\kappa(A) := \|A\|\|A^+\| 
$$

Notice that if $A$ is invertible, then in-fact $A^+$ reduces to $A^{-1}$. So, this is a natural extension of the condition number definition. 

## Exercises

### Exercise 1

Consider the problem of solving $Ax=b$ in the least-squares sense where 

$$
A = \begin{bmatrix}
1 &1 \\
1 &1.0001 \\
1 &1.0001 
\end{bmatrix}, \; \; \; \text{ and } \; \; \; 
b = \begin{bmatrix}
2 \\
.0001 \\
4.0001 
\end{bmatrix}
$$

a) Calculate $A^+$ for the example above. Give exact answers. 
<br>
b) Find the exact solution to $Ax=b$ by minimizing the residual under least squares 
<br>
c) Calculate $\kappa_\infty(A)$ and $||A||_F$. Give exact answers. 


### Exercise 2

Let $u$ and $v$ be two vectors and let $E$ be the *outer product*, i.e.

$$
E = u v^T = \begin{bmatrix}
u_1 v_1 &u_2 v_1 &... &u_n v_1 \\
u_1 v_2 &u_2 v_2 &... &u_n v_2 \\
\vdots & &\ddots &\vdots \\
u_1 v_n &u_2 v_n &... &u_n v_n
\end{bmatrix}
$$

Is it true that $\|E\|_F = \|u\|_F \|v\|_F$? Prove it or give a counterexample. 